<a href="https://colab.research.google.com/github/hassiahk/Double-Hard-Debias/blob/main/notebooks/Qualitative_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import codecs
import os
import json
import random
import warnings
from typing import Dict, List

import numpy as np
import scipy.spatial

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything()

In [ ]:
from double_hard_debias.utils import load_glove_txt


glove, glove_word2idx, vocab = load_glove_txt("../data/vectors.txt")
dhd_glove_reproduced, dhd_glove_word2idx_reproduced, dhd_vocab_reproduced = load_glove_txt("../data/glove_dhd.txt")

322636 (322636, 300) 322636
322636 (322636, 300) 322636


### Restrict the vocabulary

In [ ]:
from tqdm import tqdm
from double_hard_debias.utils import limit_vocab


gender_specific_words = []

with open('./data/male_word_file.txt') as f:
    for l in f:
        gender_specific_words.append(l.strip())
with open('./data/female_word_file.txt') as f:
    for l in f:
        gender_specific_words.append(l.strip())
print(len(gender_specific_words))

with codecs.open('./data/gender_specific_full.json') as f:
    gender_specific_words.extend(json.load(f))

print(len(gender_specific_words))

442
1883


In [ ]:
glove_limit, glove_word2idx_limit, glove_vocab_limit = limit_vocab(
    glove, glove_word2idx, vocab, exclude=gender_specific
)
dhd_limit, dhd_word2idx_limit, dhd_vocab_limit = limit_vocab(
    dhd_glove_reproduced, dhd_glove_word2idx_reproduced, dhd_vocab_reproduced, exclude=gender_specific
)

  0%|          | 0/50000 [00:00<?, ?it/s]

size of vocabulary: 47628


100%|██████████| 50000/50000 [00:00<00:00, 394301.56it/s]


size of vocabulary: 47628


### Compute bias before and after debiasing by taking the difference between consine similarity with he and she vector.

In [ ]:
def cosine_similarity(word_vec1: np.ndarray, word_vec2: np.ndarray) -> float:
    """
    Computes Cosine Similarity between two word embeddings or vectors.
    Args:
        word_vec1: Word vector of a word.
        word_vec2: Word vector of a word.
    Returns:
        Cosine similarity coefficient.
    """
    return 1 - scipy.spatial.distance.cosine(word_vec1, word_vec2)

def compute_bias_by_projection(
    word_vec: np.ndarray,
    word2idx: Dict[str, int],
    vocab: List[str],
    he_embed: np.ndarray,
    she_embed: np.ndarray,
) -> Dict[str, float]:
    """
    Computes bias of each word by taking the difference of the word's similarity to `he` word embedding
    and the word's similarity to `she` word embedding.
    Args:
        word_vec: Word vector of different words.
        word2idx: Word to index mapping.
        vocab: Vocabulary.
    """
    gender_bias: Dict[str, float] = {}

    for word in vocab:
        vector = word_vec[word2idx[word]]
        gender_bias[word] = cosine_similarity(vector, he_embed) - cosine_similarity(vector, she_embed)

    return gender_bias

In [ ]:
he_embed = glove[glove_word2idx['he'], :]
she_embed = glove[glove_word2idx['she'], :]

gender_bias_before = compute_bias_by_projection(
    glove_limit, glove_word2idx_limit, glove_vocab_limit, he_embed, she_embed
)

In [ ]:
he_embed_dhd = dhd_glove_reproduced[dhd_glove_word2idx_reproduced['he'], :]
she_embed_dhd = dhd_glove_reproduced[dhd_glove_word2idx_reproduced['she'], :]

gender_bias_after = compute_bias_by_projection(
    dhd_limit, dhd_word2idx_limit, dhd_vocab_limit, he_embed_dhd, she_embed_dhd
)

### Gender bias before and after debiasing. Positive means that the word is biased towards he and negative means that the word is biased towards she.

In [ ]:
def bias_before_after(word: str):
    print(f'Word: {word}')
    print(f'Bias Before: {gender_bias_before[word]}\nBias After: {gender_bias_after[word]}')

In [ ]:
bias_before_after('doctor')

Word: doctor
Bias Before: 0.012779653811138436
Bias After: 0.009506828053698513


In [ ]:
bias_before_after('programmer')

Word: programmer
Bias Before: 0.03555772823044012
Bias After: -0.007160397265488783


In [ ]:
bias_before_after('homemaker')

Word: homemaker
Bias Before: -0.1121882437408126
Bias After: 0.03256866170158712


In [ ]:
bias_before_after('nurse')

Word: nurse
Bias Before: -0.12097890805438682
Bias After: 0.03308098877483401


In [ ]:
bias_before_after('worker')

Word: worker
Bias Before: -0.006665410002464012
Bias After: 0.023414286810958918


In [ ]:
bias_before_after('president')

Word: president
Bias Before: 0.08256089765387709
Bias After: 0.03410448077336892


In [ ]:
bias_before_after('politician')

Word: politician
Bias Before: 0.06632057738561792
Bias After: 0.02851813613094467
